In [1]:
import os
import glob
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


In [2]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("MyApp") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .master("local[*]") \
    .getOrCreate()
# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/31 10:22:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Set up config

In [28]:
# set up temp config for trial run
model_train_date_str = "2017-03-01"
train_test_period_months = 0
oot_period_months = 0
train_test_ratio = 0.8

config = {}
config["model_train_date_str"] = model_train_date_str
config["train_test_period_months"] = train_test_period_months
config["oot_period_months"] =  oot_period_months
config["model_train_date"] =  datetime.strptime(model_train_date_str, "%Y-%m-%d")
config["oot_end_date"] =  config['model_train_date'] - timedelta(days = 0)
config["oot_start_date"] =  config['model_train_date'] - relativedelta(months = oot_period_months)
config["train_test_end_date"] =  config["oot_start_date"] - timedelta(days = 0)
config["train_test_start_date"] =  config["oot_start_date"] - relativedelta(months = train_test_period_months)
config["train_test_ratio"] = train_test_ratio 

pprint.pprint(config)

{'model_train_date': datetime.datetime(2017, 3, 1, 0, 0),
 'model_train_date_str': '2017-03-01',
 'oot_end_date': datetime.datetime(2017, 3, 1, 0, 0),
 'oot_period_months': 0,
 'oot_start_date': datetime.datetime(2017, 3, 1, 0, 0),
 'train_test_end_date': datetime.datetime(2017, 3, 1, 0, 0),
 'train_test_period_months': 0,
 'train_test_ratio': 0.8,
 'train_test_start_date': datetime.datetime(2017, 3, 1, 0, 0)}


## set up config
model_train_date_str = "2017-03-01"
train_test_period_months = 12
oot_period_months = 2
train_test_ratio = 0.8

config = {}
config["model_train_date_str"] = model_train_date_str
config["train_test_period_months"] = train_test_period_months
config["oot_period_months"] =  oot_period_months
config["model_train_date"] =  datetime.strptime(model_train_date_str, "%Y-%m-%d")
config["oot_end_date"] =  config['model_train_date'] - timedelta(days = 1)
config["oot_start_date"] =  config['model_train_date'] - relativedelta(months = oot_period_months)
config["train_test_end_date"] =  config["oot_start_date"] - timedelta(days = 1)
config["train_test_start_date"] =  config["oot_start_date"] - relativedelta(months = train_test_period_months)
config["train_test_ratio"] = train_test_ratio 


pprint.pprint(config)

## Get Label Store

In [4]:
# connect to label store
folder_path = "/app/datamart/gold/label_store/"
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
label_store_sdf = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",label_store_sdf.count())

label_store_sdf.show()

row_count: 20146895
+-------------+--------------------+--------+
|snapshot_date|                msno|is_churn|
+-------------+--------------------+--------+
|   2017-02-20|++0/NopttBsaAn6qH...|       0|
|   2016-04-20|++0/NopttBsaAn6qH...|       0|
|   2016-11-20|++0/NopttBsaAn6qH...|       0|
|   2016-09-20|++0/NopttBsaAn6qH...|       0|
|   2016-05-20|++0/NopttBsaAn6qH...|       0|
|   2016-07-20|++0/NopttBsaAn6qH...|       0|
|   2016-08-20|++0/NopttBsaAn6qH...|       0|
|   2017-01-20|++0/NopttBsaAn6qH...|       0|
|   2016-12-20|++0/NopttBsaAn6qH...|       0|
|   2016-06-20|++0/NopttBsaAn6qH...|       0|
|   2016-10-20|++0/NopttBsaAn6qH...|       0|
|   2016-12-02|++0GCV3WGMjibrwCn...|       1|
|   2016-02-20|++3hfQtTKeHLVuBHI...|       1|
|   2015-11-20|++3hfQtTKeHLVuBHI...|       0|
|   2015-12-20|++3hfQtTKeHLVuBHI...|       0|
|   2016-01-20|++3hfQtTKeHLVuBHI...|       0|
|   2016-02-15|++3hfQtTKeHLVuBHI...|       1|
|   2016-12-13|++6P09mCSJSh+Ft2p...|       0|
|   2015-09-13

In [5]:
# extract label store
labels_sdf = label_store_sdf.filter((col("snapshot_date") >= config["train_test_start_date"]) & (col("snapshot_date") <= config["oot_end_date"]))

print("extracted labels_sdf", labels_sdf.count(), config["train_test_start_date"], config["oot_end_date"])

[Stage 6:>                                                          (0 + 8) / 9]

extracted labels_sdf 39544 2017-03-01 00:00:00 2017-03-01 00:00:00


## Get Feature Store 

In [6]:
feature_location = "/app/datamart/gold/feature_store/"

In [7]:
# Load CSV into DataFrame - connect to feature store
features_file_list = [feature_location+os.path.basename(f) for f in glob.glob(os.path.join(feature_location, '*'))]

features_store_sdf = spark.read \
                          .option("header", True) \
                          .option("inferSchema", True) \
                          .parquet(*features_file_list)

print("row_count:",features_store_sdf.count())

features_store_sdf.show()

row_count: 37254052
+--------------------+-----------------+-----------------------+--------------+----------+--------------+---------------+------------------+---------------+------------------+------------------+---------------------+--------------+--------------------+-------------------+-----------------------+-----------+----------------+------------------+
|                msno|registration_date|tenure_days_at_snapshot|registered_via|city_clean|        via_oh|        city_oh|      sum_secs_w30|active_days_w30| complete_rate_w30|       sum_secs_w7|engagement_ratio_7_30|last_play_date|days_since_last_play|     trend_secs_w30|latest_transaction_date|tenure_days|auto_renew_share|last_is_auto_renew|
+--------------------+-----------------+-----------------------+--------------+----------+--------------+---------------+------------------+---------------+------------------+------------------+---------------------+--------------+--------------------+-------------------+------------------

In [8]:
features_store_sdf = features_store_sdf.withColumn(
    "snapshot_date",
    F.to_date(F.lit("2017-03-01"), "yyyy-MM-dd")
)

In [9]:
# extract feature store
features_sdf = features_store_sdf.filter((col("snapshot_date") >= config["train_test_start_date"]) & (col("snapshot_date") <= config["oot_end_date"]))

print("extracted features_sdf", features_sdf.count(), config["train_test_start_date"], config["oot_end_date"])

[Stage 15:====================================================>   (48 + 3) / 51]

extracted features_sdf 37254052 2017-03-01 00:00:00 2017-03-01 00:00:00


## Prepare Data for Modelling

In [10]:
# prepare data for modeling
data_pdf = labels_sdf.join(features_sdf, on=["msno", "snapshot_date"], how="left").toPandas()
data_pdf = data_pdf.rename(columns={"is_churn": "label"})
data_pdf

,msno,snapshot_date,label,registration_date,tenure_days_at_snapshot,registered_via,city_clean,via_oh,city_oh,sum_secs_w30,...,complete_rate_w30,sum_secs_w7,engagement_ratio_7_30,last_play_date,days_since_last_play,trend_secs_w30,latest_transaction_date,tenure_days,auto_renew_share,last_is_auto_renew
0,+0sDKWd6EDcu1roUYhvYHeEDUatpUtav6j3AzRGNTVA=,2017-03-01,0,2016-07-01,243.0,7,1,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",660.374,...,0.500000,0.000,0.000000,2017-02-09,20.0,0.000000,2017-03-01,243.0,1.0,1.0
1,+0sDKWd6EDcu1roUYhvYHeEDUatpUtav6j3AzRGNTVA=,2017-03-01,0,2016-07-01,243.0,7,1,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",660.374,...,0.500000,0.000,0.000000,2017-02-09,20.0,0.000000,2017-03-01,243.0,1.0,1.0
2,+0sDKWd6EDcu1roUYhvYHeEDUatpUtav6j3AzRGNTVA=,2017-03-01,0,2016-07-01,215.0,7,1,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000,...,0.000000,0.000,0.000000,2016-12-30,33.0,0.000000,2017-02-01,215.0,1.0,1.0
3,+0sDKWd6EDcu1roUYhvYHeEDUatpUtav6j3AzRGNTVA=,2017-03-01,0,2016-07-01,184.0,7,1,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3690.212,...,0.882353,2681.135,0.726553,2016-12-30,2.0,1672.058000,2017-01-01,184.0,1.0,1.0
4,+0sDKWd6EDcu1roUYhvYHeEDUatpUtav6j3AzRGNTVA=,2017-03-01,0,2016-07-01,153.0,7,1,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",8961.544,...,0.878049,0.000,0.000000,2016-11-16,15.0,-165.085000,2016-12-01,153.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221909,zzVuAn+JQ/5X80yCTD4ZcnEWZ709xhzD9++UtUovUGw=,2017-03-01,0,2014-01-07,1149.0,7,1,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",82076.321,...,1.079872,1271.586,0.015493,2017-02-25,4.0,51.980769,2017-03-01,1149.0,1.0,1.0
221910,zzVuAn+JQ/5X80yCTD4ZcnEWZ709xhzD9++UtUovUGw=,2017-03-01,0,2014-01-07,1149.0,7,1,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",82076.321,...,1.079872,1271.586,0.015493,2017-02-25,4.0,51.980769,2017-03-01,1149.0,1.0,1.0
221911,zzVuAn+JQ/5X80yCTD4ZcnEWZ709xhzD9++UtUovUGw=,2017-03-01,0,2014-01-07,1121.0,7,1,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",55607.326,...,1.067358,10508.696,0.188980,2017-02-01,0.0,-327.654608,2017-02-01,1121.0,1.0,1.0
221912,zzVuAn+JQ/5X80yCTD4ZcnEWZ709xhzD9++UtUovUGw=,2017-03-01,0,2014-01-07,1090.0,7,1,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",127105.656,...,1.771218,13960.884,0.109837,2016-12-30,2.0,-612.750411,2017-01-01,1090.0,1.0,1.0


In [14]:
features_sdf.columns

['msno',
 'registration_date',
 'tenure_days_at_snapshot',
 'registered_via',
 'city_clean',
 'via_oh',
 'city_oh',
 'sum_secs_w30',
 'active_days_w30',
 'complete_rate_w30',
 'sum_secs_w7',
 'engagement_ratio_7_30',
 'last_play_date',
 'days_since_last_play',
 'trend_secs_w30',
 'latest_transaction_date',
 'tenure_days',
 'auto_renew_share',
 'last_is_auto_renew',
 'snapshot_date']

In [16]:
# split data into train - test - oot
oot_pdf = data_pdf[(data_pdf['snapshot_date'] >= config["oot_start_date"].date()) & (data_pdf['snapshot_date'] <= config["oot_end_date"].date())]
train_test_pdf = data_pdf[(data_pdf['snapshot_date'] >= config["train_test_start_date"].date()) & (data_pdf['snapshot_date'] <= config["train_test_end_date"].date())]

#feature_cols = [fe_col for fe_col in data_pdf.columns if fe_col.startswith('fe_')]
feature_cols = ['tenure_days_at_snapshot',
 'registered_via',
 'city_clean', 
'sum_secs_w30',
 'active_days_w30',
 'complete_rate_w30',
 'sum_secs_w7',
 'engagement_ratio_7_30',
 'days_since_last_play',
 'trend_secs_w30','auto_renew_share',
 'last_is_auto_renew']
                
X_oot = oot_pdf[feature_cols]
y_oot = oot_pdf["label"]
X_train, X_test, y_train, y_test = train_test_split(
    train_test_pdf[feature_cols], train_test_pdf["label"], 
    test_size= 1 - config["train_test_ratio"],
    random_state=88,     # Ensures reproducibility
    shuffle=True,        # Shuffle the data before splitting
    stratify=train_test_pdf["label"]           # Stratify based on the label column
)


print('X_train', X_train.shape[0])
print('X_test', X_test.shape[0])
print('X_oot', X_oot.shape[0])
print('y_train', y_train.shape[0], round(y_train.mean(),2))
print('y_test', y_test.shape[0], round(y_test.mean(),2))
print('y_oot', y_oot.shape[0], round(y_oot.mean(),2))

X_train

X_train 177531
X_test 44383
X_oot 221914
y_train 177531 0.07
y_test 44383 0.07
y_oot 221914 0.07


,tenure_days_at_snapshot,registered_via,city_clean,sum_secs_w30,active_days_w30,complete_rate_w30,sum_secs_w7,engagement_ratio_7_30,days_since_last_play,trend_secs_w30,auto_renew_share,last_is_auto_renew
191308,2119.0,7,8,28726.972,11.0,0.915385,1276.980,0.044452,4.0,-410.943245,0.00,0.0
99689,365.0,7,1,0.000,0.0,0.000000,0.000,0.000000,353.0,0.000000,1.00,1.0
168373,1902.0,7,13,5691.878,10.0,0.846154,242.787,0.042655,6.0,-28.114945,1.00,1.0
145114,791.0,7,8,45610.904,8.0,4.578947,14327.815,0.314131,1.0,-412.336786,1.00,1.0
72001,1787.0,7,1,94419.815,20.0,0.866142,35007.110,0.370760,0.0,148.382665,1.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
164505,1226.0,3,15,93216.236,13.0,1.261194,27304.138,0.292912,0.0,-147.852593,1.00,1.0
175672,1079.0,7,4,51443.230,17.0,0.932367,6692.741,0.130100,1.0,-155.828091,1.00,1.0
57899,1614.0,3,15,73006.443,20.0,0.775811,12901.268,0.176714,0.0,11.466201,0.56,1.0
94027,275.0,7,1,27241.848,12.0,0.830508,12278.181,0.450710,0.0,284.714888,1.00,1.0


## Preprocess Data

In [17]:
# set up standard scalar preprocessing
scaler = StandardScaler()

transformer_stdscaler = scaler.fit(X_train) 

# transform data
X_train_processed = transformer_stdscaler.transform(X_train)
X_test_processed = transformer_stdscaler.transform(X_test)
X_oot_processed = transformer_stdscaler.transform(X_oot)

print('X_train_processed', X_train_processed.shape[0])
print('X_test_processed', X_test_processed.shape[0])
print('X_oot_processed', X_oot_processed.shape[0])

pd.DataFrame(X_train_processed)

X_train_processed 177531
X_test_processed 44383
X_oot_processed 221914


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.823266,0.028551,0.268576,-0.393982,0.083051,0.011077,-0.696834,-0.696768,-0.221215,-0.362773,-2.371962,-2.372487
1,-0.924181,0.028551,-0.797958,-0.959336,-1.320140,-0.830634,-0.790505,-0.903058,4.278206,0.009449,0.457980,0.421499
2,0.607076,0.028551,1.030385,-0.847318,-0.044512,-0.052581,-0.772696,-0.705109,-0.195430,-0.016017,0.457980,0.421499
3,-0.499773,0.028551,0.268576,-0.061702,-0.299638,3.379783,0.260492,0.554734,-0.259892,-0.364035,0.457980,0.421499
4,0.492506,0.028551,-0.797958,0.898870,1.231116,-0.034202,1.777394,0.817532,-0.272784,0.143851,0.457980,0.421499
...,...,...,...,...,...,...,...,...,...,...,...,...
177526,-0.066398,-2.113031,1.335109,0.875183,0.338176,0.329055,1.212353,0.456260,-0.272784,-0.124472,0.457980,0.421499
177527,-0.212849,0.028551,-0.340872,0.053080,0.848427,0.026693,-0.299568,-0.299304,-0.259892,-0.131696,0.457980,0.421499
177528,0.320152,-2.113031,1.335109,0.477449,1.231116,-0.117263,0.155850,-0.082979,-0.272784,0.019835,-0.787194,0.421499
177529,-1.013845,0.028551,-0.797958,-0.423210,0.210613,-0.066967,0.110144,1.188557,-0.272784,0.267337,0.457980,0.421499


## Train Model

#### Logistics Regression

#### XGBoost

In [18]:
# Define the XGBoost classifier
xgb_clf = xgb.XGBClassifier(eval_metric='logloss', random_state=88)

# Define the hyperparameter space to search
param_dist = {
    'n_estimators': [25, 50],
    'max_depth': [2, 3],  # lower max_depth to simplify the model
    'learning_rate': [0.01, 0.1],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 0.8],
    'gamma': [0, 0.1],
    'min_child_weight': [1, 3, 5],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2]
}

# Create a scorer based on AUC score
auc_scorer = make_scorer(roc_auc_score)

# Set up the random search with cross-validation
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    scoring=auc_scorer,
    n_iter=10,  # Number of iterations for random search
    cv=3,       # Number of folds in cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Perform the random search
random_search.fit(X_train_processed, y_train)

# Output the best parameters and best score
print("Best parameters found: ", random_search.best_params_)
print("Best AUC score: ", random_search.best_score_)

# Evaluate the model on the train set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_train_processed)[:, 1]
train_auc_score = roc_auc_score(y_train, y_pred_proba)
print("Train AUC score: ", train_auc_score)

# Evaluate the model on the test set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test_processed)[:, 1]
test_auc_score = roc_auc_score(y_test, y_pred_proba)
print("Test AUC score: ", test_auc_score)

# Evaluate the model on the oot set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_oot_processed)[:, 1]
oot_auc_score = roc_auc_score(y_oot, y_pred_proba)
print("OOT AUC score: ", oot_auc_score)

print("TRAIN GINI score: ", round(2*train_auc_score-1,3))
print("Test GINI score: ", round(2*test_auc_score-1,3))
print("OOT GINI score: ", round(2*oot_auc_score-1,3))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters found:  {'subsample': 0.6, 'reg_lambda': 2, 'reg_alpha': 1, 'n_estimators': 50, 'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 0.8}
Best AUC score:  0.5063781075356247
Train AUC score:  0.7721200623005164
Test AUC score:  0.7603447039324496
OOT AUC score:  0.7697662038240716
TRAIN GINI score:  0.544
Test GINI score:  0.521
OOT GINI score:  0.54


## Prepare model artefact to save

In [30]:
model_artefact = {}

model_artefact['model'] = best_model
model_artefact['model_version'] = "credit_model_" + config['model_train_date_str'].replace('-','_')
model_artefact['preprocessing_transformers'] = {}
model_artefact['preprocessing_transformers']['stdscaler'] = transformer_stdscaler
model_artefact['data_dates'] = config
model_artefact['data_stats'] = {}
model_artefact['data_stats']['X_train'] = X_train.shape[0]
model_artefact['data_stats']['X_test'] = X_test.shape[0]
model_artefact['data_stats']['X_oot'] = X_oot.shape[0]
model_artefact['data_stats']['y_train'] = round(y_train.mean(),2)
model_artefact['data_stats']['y_test'] = round(y_test.mean(),2)
model_artefact['data_stats']['y_oot'] = round(y_oot.mean(),2)
model_artefact['results'] = {}
model_artefact['results']['auc_train'] = train_auc_score
model_artefact['results']['auc_test'] = test_auc_score
model_artefact['results']['auc_oot'] = oot_auc_score
model_artefact['results']['gini_train'] = round(2*train_auc_score-1,3)
model_artefact['results']['gini_test'] = round(2*test_auc_score-1,3)
model_artefact['results']['gini_oot'] = round(2*oot_auc_score-1,3)
model_artefact['hp_params'] = random_search.best_params_


pprint.pprint(model_artefact)

{'data_dates': {'model_train_date': datetime.datetime(2017, 3, 1, 0, 0),
                'model_train_date_str': '2017-03-01',
                'oot_end_date': datetime.datetime(2017, 3, 1, 0, 0),
                'oot_period_months': 0,
                'oot_start_date': datetime.datetime(2017, 3, 1, 0, 0),
                'train_test_end_date': datetime.datetime(2017, 3, 1, 0, 0),
                'train_test_period_months': 0,
                'train_test_ratio': 0.8,
                'train_test_start_date': datetime.datetime(2017, 3, 1, 0, 0)},
 'data_stats': {'X_oot': 221914,
                'X_test': 44383,
                'X_train': 177531,
                'y_oot': np.float64(0.07),
                'y_test': np.float64(0.07),
                'y_train': np.float64(0.07)},
 'hp_params': {'colsample_bytree': 0.8,
               'gamma': 0,
               'learning_rate': 0.1,
               'max_depth': 3,
               'min_child_weight': 5,
               'n_estimators': 50,
        

## Save Artefact to Model Bank

In [31]:
# create model_bank dir
model_bank_directory = "model_bank/"

if not os.path.exists(model_bank_directory):
    os.makedirs(model_bank_directory)

In [32]:
# Full path to the file
file_path = os.path.join(model_bank_directory, model_artefact['model_version'] + '.pkl')

# Write the model to a pickle file
with open(file_path, 'wb') as file:
    pickle.dump(model_artefact, file)

print(f"Model saved to {file_path}")


Model saved to model_bank/credit_model_2017_03_01.pkl


## Test Load Pickle and Make Model Inference

In [33]:
# Load the model from the pickle file
with open(file_path, 'rb') as file:
    loaded_model_artefact = pickle.load(file)

y_pred_proba = loaded_model_artefact['model'].predict_proba(X_oot_processed)[:, 1]
oot_auc_score = roc_auc_score(y_oot, y_pred_proba)
print("OOT AUC score: ", oot_auc_score)

print("Model loaded successfully!")

OOT AUC score:  0.7697662038240716
Model loaded successfully!


In [34]:
spark.stop()